# 11장 인공지능 비서, 자비스 프로젝트

## 11.1 학습목표 
- 마크 저커버그의 자비스 영상 : https://youtu.be/ZGLPxEv_EWo 

## 11.2 사전 준비 하기
- SpeechRecognition 라이브러리 문서 : https://pypi.org/project/SpeechRecognition/
- pyaudio wheel 파일 다운 : www.lfd.uci.edu/~gohlke/pythonlibs/#pyaudio

- !pip install SpeechRecognition
- !pip install <다운로드받은 .whl 파일명>

## 11.3 사전 지식 쌓기

### 11.3.1 음성인식(STT) 하기

In [ ]:
import requests
import json

# 함수 정의부
def kakao_stt(app_key, stype, data):
    if stype == 'file':
        filename = data
        with open(filename, "rb") as fp:
            audio = fp.read()
    else:
        audio =data

    headers = {
        "Content-Type": "application/octet-stream",
        "Authorization": "KakaoAK " + app_key,
    }

    # 카카오 음성 url
    kakao_speech_url = "https://kakaoi-newtone-openapi.kakao.com/v1/recognize"
    # 카카오 음성 api 요청
    res = requests.post(kakao_speech_url, headers=headers, data=audio)
    # 요청에 실패했다면,
    if res.status_code != 200:
        text=""
        print("error! because ", res.json())
    else: # 성공했다면,
        #print("음성인식 결과 : ", res.text)
        #print("시작위치 : ", res.text.index('{"type":"finalResult"'))
        #print("종료위치 : ", res.text.rindex('}')+1)
        #print("추출한 정보 : ", res.text[res.text.index('{"type":"finalResult"'):res.text.rindex('}')+1])
        result = res.text[res.text.index('{"type":"finalResult"'):res.text.rindex('}')+1]
        text = json.loads(result).get('value')

    return text

# 함수 호출부
KAKAO_APP_KEY = "<REST_API 앱키를 입력하세요>"
AUDIO_FILE = "res/jarvis/hello.wav"
text = kakao_stt(KAKAO_APP_KEY, "file", AUDIO_FILE)
print(text)

## 11.3.2 마이크(mic)로 음성 수집하기

In [ ]:
import speech_recognition as sr

# 함수 정의부
def get_speech():
    # 마이크에서 음성을 추출하는 객체
    recognizer = sr.Recognizer()

    # 마이크 설정
    microphone = sr.Microphone(sample_rate=16000)

    # 마이크 소음 수치 반영
    with microphone as source:
        recognizer.adjust_for_ambient_noise(source)
        print("소음 수치 반영하여 음성을 청취합니다. {}".format(recognizer.energy_threshold))

    # 음성 수집
    with microphone as source:
        print("Say something!")
        result = recognizer.listen(source)
        audio = result.get_raw_data()

    return audio

# 함수 호출부
audio = get_speech()
text = kakao_stt(KAKAO_APP_KEY, "stream", audio)
print("음성 인식 결과 : " + text)

# 11.4 구현하기

## Step1) 구현 코드 정리 하기 - '날씨'

In [ ]:
# 정상 동작 확인용
import jarvis_food_recommender
jarvis_food_recommender.do()

## Step2) 구현 코드 정리하기 - '주식'

In [ ]:
# 정상 동작 확인용
import jarvis_stock_report
jarvis_stock_report.do()

## Step3) 자비스 프로젝트

In [ ]:
import jarvis_food_recommender
import jarvis_stock_report

KAKAO_APP_KEY = "<REST_API 앱키를 입력하세요>"

KEYWORD_STOCK = '주식'
KEYWORD_WEATHER = '날씨'

# 음성 수집
audio = get_speech()

# STT
command = kakao_stt(KAKAO_APP_KEY, "stream", audio)
print("명령어 : " + command)
print('명령을 수행합니다.')

# 음성 분석
if KEYWORD_WEATHER in command:
    print('날씨를 파악하여 맛집을 추천합니다.')
    jarvis_food_recommender.do()
elif KEYWORD_STOCK in command:
    print('주식 보고서를 메일로 전송합니다.')
    jarvis_stock_report.do()
else:
    print('명령을 알 수 없습니다.')